In [1]:
import os
import numpy as np
import cv2
from numpy import unravel_index
import math
import matplotlib.pyplot as plt

In [2]:
def FFT(image):
    # Return the fast fourier transform after it is centered
    fft = np.fft.fft2(image, s=None, axes=(-2, -1), norm=None)
    fft = np.fft.fftshift(fft, axes=None)
    return fft

def power_spectrum(fft):
    # Returns the power spectrum of the 2D-Fourier Transform
    return np.abs(fft)**2

def findPeakGrid(a):
    height = len(a)
    width = len(a[0])
    b = np.asarray(a)
    
    return unravel_index(b.argmax(), b.shape)


def maskDots(fft_img, X, Y, radius_squared):
    # Compute the squared distances from the circle center
    r1 = (np.arange(crop_size)[:, None] - Y - crop_size/2) ** 2 + (np.arange(crop_size) - X - crop_size/2) ** 2
    r2 = (np.arange(crop_size)[:, None] + Y - crop_size/2) ** 2 + (np.arange(crop_size) + X - crop_size/2) ** 2

    
    # Create a mask that is True where both distances are greater than radius ** 2
    mask = np.logical_and(r1 > radius_squared, r2 > radius_squared)

    # Apply the mask to the FFT image
    fft_img[mask] = 0
    
    fft_copy = fft_img.copy()

    return fft_copy

def showImage(image, X, Y, radius_squared):
    # Takes a numpy array and transforms it into cv2 heatmap
    
    # Logarithmic and normalize
    image_log = np.log(np.abs(image.astype(np.float64)))
    image_norm = cv2.normalize(image_log, None, 0, 255, cv2.NORM_MINMAX)
    
    # Grayscale from normalized logarithmic image
    image_gray = np.uint8(image_norm)*255
    
    # Create heatmap from grayscale
    image_heatmap = cv2.applyColorMap(image_gray, cv2.COLORMAP_JET)
    
    # Add circle to show where mask is
    w = len(image_heatmap)
    h = len(image_heatmap[0])
    image_heatmap = cv2.circle(image_heatmap, (int(w/2+X),int(h/2+Y)), int(math.sqrt(radius_squared)), (0, 0, 0), 1)
    #cv2.imshow("circle",image_heatmap)
    
    # Resize image
    image_heatmap = cv2.resize(image_heatmap, (w*4, h*4))

    # Crop unnecessary parts out
    w = len(image_heatmap)
    h = len(image_heatmap[0])
    image_heatmap = image_heatmap[int(w/2)-120:int(w/2)+120, int(h/2)-120:int(h/2)+120]
    
    # Image to contain heatmap color bar gradient
    color_bar = np.zeros((image_heatmap.shape[0], 70, 3), dtype=np.uint8)
    gradient = np.linspace(0, 255, image_heatmap.shape[0]).astype(np.uint8)
    
    # Add the gradient to the color bar
    for i, intensity in enumerate(gradient):
        color = cv2.applyColorMap(np.array([[intensity]]), cv2.COLORMAP_JET)[0][0]
        color_bar[i] = color
    
    # Concatenate the color bar and the heatmap horizontally
    image_with_colorbar = np.concatenate((image_heatmap, color_bar), axis=1)
    
    # Settings for color bar font and placement
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.4
    font_color = (255, 255, 255)  # White
    font_thickness = 1
    font_color2 = (0,0,0)  # White
    font_thickness2 = 2
    value_step = len(image_heatmap) / 5.5

    # Actual numeric values
    image_minvalues = np.amin(image_log)
    image_maxvalues = np.amax(image_log)
    
    color_bar_values = np.exp(np.linspace(image_minvalues, image_maxvalues, num=6))
    
    for i in range(6):
        value = color_bar_values[i]
        cv2.putText(
            image_with_colorbar, 
            "{:.1e}".format(value), 
            (image_with_colorbar.shape[1]-60, image_with_colorbar.shape[0] - int(i * value_step) - 10),
            font, 
            font_scale, 
            font_color2, 
            font_thickness2)
        cv2.putText(
            image_with_colorbar, 
            "{:.1e}".format(value), 
            (image_with_colorbar.shape[1]-60, image_with_colorbar.shape[0] - int(i * value_step) - 10),
            font, 
            font_scale, 
            font_color, 
            font_thickness)
        
    return image_with_colorbar

In [3]:
# path = "C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/testResults"
path = "C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/testResults2"
crop_size = 300

In [ ]:
for filename in os.listdir(path):
    if filename.endswith(".png"):
        file_path = os.path.join(path, filename)
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = image[0:crop_size, 0:crop_size]
        
        fft = FFT(image)
        ps = power_spectrum(fft)
        #size = int(crop_size/2)
        #ps[size-4:size+4, size-4:size+4] = 0

        text = filename
        text = filename.split(".")
        text = text[0].split("_")
        
        # testresults
#         radius_squared = 10
#         radius_dots = 3*int(text[0])/10-9
#         angle = int(text[1])
#         X = radius_dots * math.sin(math.pi * 2 * angle / 360 + math.pi/2);
#         Y = radius_dots * math.cos(math.pi * 2 * angle / 360 + math.pi/2);
        
        # testresults2
        radius_squared = 17
        radius_dots = 50*int(text[0])**0.5/10-28
        angle = int(text[1])
        X = radius_dots * math.sin(math.pi * 2 * angle / 360 + math.pi/2);
        Y = radius_dots * math.cos(math.pi * 2 * angle / 360 + math.pi/2);
        
        ps_mask = ps.copy()
        ps_mask = maskDots(ps_mask, X, Y, radius_squared)
        #ps_mask_imshow = np.log(np.abs(ps_mask.copy().astype(np.float64)))
        #cv2.imshow(filename+"1", cv2.normalize(np.uint8(ps_mask_imshow)*255, None, 0, 255, cv2.NORM_MINMAX))
        
        
        avgval = np.mean(ps_mask, axis=1)
        avgval = np.mean(avgval)
        #print(avgval)
        
        maxval = np.amax(ps_mask)
        #print(maxval)
        #print('{:.1e}'.format(maxval))
        
        maxvalpos = findPeakGrid(ps_mask)
        
        #print("mm", text)        
        #print("mm", text[0], ": deg", text[1])        
        print(int(avgval))
        #print(maxvalpos)
        
        # Show image
        cv2.imshow(filename, showImage(ps, X, Y, radius_squared))
        
cv2.waitKey(0) #wait for any key
cv2.destroyAllWindows() #close the image window

1837848
1337899
30140
10392
9463
15451
43231
95811
1334897
1859007
4742787
4402504
1426999
1234101
1440879
1816118
2313207
3151598
6527010
7001033
15779017
13800473
10500170
5758912
6211043
7384521
8822036
15674827
20128694
20296459
59263023
57386489
56667879
56825903
36555519
37190354
57961645
63692045
64338140
67883910
